In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from google.colab import files
import ipywidgets as widgets



# upload CSV file
uploaded = files.upload()

# load dataset
df = pd.read_csv(next(iter(uploaded)))

#use pokemon_sets.csv

Saving newdata.csv to newdata.csv


# Choice of Libraries Explanation:
# The following libraries were chosen:

# Pandas: Used for data manipulation and analysis. The DataFrame structure simplifies working with tabular data.
# Scikit-Learn: Cosine_similarity function is used for calculating cosine similarity between vectors.
# PrettyTable: Used for presenting tabular data in a visually appealing format.
# Matplotlib: While not used in the provided code, it's a powerful library for creating visualizations if needed.
# Ipywidgets: Enables the creation of interactive widgets for user input, enhancing the user experience.

In [ ]:
# define the features to consider for similarity calculation
features = ['name', 'item', 'ability', 'tera_type', 'hp_evs', 'atk_evs', 'def_evs', 'spatk_evs', 'spdef_evs', 'spe_evs', 'nature', 'move_1', 'move_2', 'move_3', 'move_4']

# replace NaN values with an empty string
df[features] = df[features].fillna('')

# combine all the features into a single column
df['combined_features'] = df[features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Use TfidfVectorizer to convert the combined features into a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

# Calculate the cosine similarity between teams
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Feature Selection and Processing Explanation:
# The selected features for similarity calculation.
# Features such as 'name', 'ability', 'move_1', etc., are crucial in capturing the essence of a Pokemon's characteristics.

# TfidfVectorizer Explanation:
# TfidfVectorizer is used to convert the combined features into TF-IDF features. Here's the reasoning:

# - TF-IDF reflects the importance of a word in a document relative to a collection of documents.
# - TF-IDF vectorization helps emphasize the importance of specific attributes that might be distinctive for certain Pokemon.
# - By converting the combined features into TF-IDF vectors, the matrix captures the uniqueness of each Pokemon based on its attributes.
# - Approach is  useful when dealing with text data because it allows the algorithm to focus on relevant information while downplaying common terms.

# Cosine Similarity Calculation Explanation:
# Cosine similarity is chosen as the metric for similarity calculation due to the following reasons:

# - Cosine similarity is well-suited for comparing the similarity of vectors in a high-dimensional space.
# - It is used in natural language processing and text analysis so it is applicable to the textual attributes of Pokemon.
# - The resulting similarity scores range from -1 to 1, where 1 indicates identical vectors.

In [ ]:

# function to get recommendations based on user input
def get_recommendations(user_input, num_recommendations=5):
    # check if the input Pokemon exists in the dataset
    user_input_pokemon = user_input.get('name', '').capitalize()
    if user_input_pokemon not in df['name'].values:
        return f"{user_input_pokemon} not found in the dataset."

    # find the index of the given Pokemon in the dataset
    index = df[df['name'] == user_input_pokemon].index[0]

    # get the cosine similarity scores for the given Pokemon
    sim_scores = list(enumerate(cosine_sim[index]))

    # sort the Pokemon based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # get the indices of the recommended Pokemon
    indices = [x[0] for x in sim_scores[1:num_recommendations+1]]

    # filter out duplicates
    unique_indices = set()
    unique_recommendations = []
    for idx in indices:
        if idx not in unique_indices:
            unique_indices.add(idx)
            unique_recommendations.append(df.iloc[idx][features])

    recommended_df = pd.DataFrame(unique_recommendations)

    if recommended_df.empty:
        return "No recommendations found based on the given input."

    return recommended_df

# function to display recommendations in a pretty table
def display_recommendations(recommended_df):
    if isinstance(recommended_df, str):
        print(recommended_df)
    else:
        table = PrettyTable(list(recommended_df.columns))
        for _, row in recommended_df.iterrows():
            table.add_row(row)
        print(table)

#button
def on_button_click(button):
    user_inputs = {key: widget.value for key, widget in user_input.items()}
    recommendations = get_recommendations(user_inputs)
    display_recommendations(recommendations)

#user input
user_input = {}
style = {'description_width': 'initial'}
user_input['name'] = widgets.Text(value='', description='Pokemon Name:', style = style)


# display the input
display(*user_input.values())

# button for submitting input
submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_button_click)
display(submit_button)

Text(value='', description='Pokemon Name:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit', style=ButtonStyle())

+----------------+--------------+------------+-----------+--------+---------+---------+-----------+-----------+---------+---------+---------------+--------------+---------------+---------------+
|      name      |     item     |  ability   | tera_type | hp_evs | atk_evs | def_evs | spatk_evs | spdef_evs | spe_evs |  nature |     move_1    |    move_2    |     move_3    |     move_4    |
+----------------+--------------+------------+-----------+--------+---------+---------+-----------+-----------+---------+---------+---------------+--------------+---------------+---------------+
|    Arcanine    | Choice Band  | Intimidate |   Normal  |   4    |   252   |    0    |     0     |     0     |   252   |  Jolly  | Extreme Speed | Flare Blitz  |   Play Rough  |  Close Combat |
|    Arcanine    | Assault Vest | Intimidate |   Grass   |  204   |   108   |    4    |     0     |     36    |   156   | Adamant |  Flare Blitz  | Close Combat | Extreme Speed |     Snarl     |
|    Arcanine    | Assaul

#Pokemon To Try:
#Arcanine
#Flutter Mane
#Palafin


# User Input Handling Explanation:
# The use of interactive widgets from ipywidgets allowis users to input a Pokemon's name directly.

# Ensures that the user input is within the dataset.


# Recommendation Function Explanation:
# The `get_recommendations` function uses cosine similarity to find Pokemon similar to the user's input.
# It checks if the input Pokemon exists in the dataset, calculates similarity scores, and returns a DataFrame with recommended Pokemon details.


# Displaying Recommendations Explanation:
# The `display_recommendations` function displays the recommendations using PrettyTable.
# This was so the information in a clear tabular format, making it easy for users to read and compare recommended Pokemon.
# If no recommendations are found,a message is displayed.


# Button Click Event Handling Explanation:.
# Collects user input, obtains recommendations, and displays them when the submit button is clicked.


# Code is organized into functions..
# Code maintainability and allows for future modifications or enhancements.

